In [2]:
# run 
# jupyter nbconvert --to script 2d_linear_confidenceintervals.ipynb
# to convert to .py strict and run the .py!
import scipy.stats as ss
import os
import numpy as np
import matplotlib.pyplot as plt
import argparse
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.svm import SVR  # for building SVR model
import pickle

import sys
sys.path.insert(0,'..')

from BMR.bmr import *
#from pyearth import Earth
import pandas as pd

In [3]:
def gen_model(X, a, b, c, d, e):
    return a*X[:, 0] + b*X[:, 1] + c*X[:, 0]**2 + d*X[:, 1]**2 + e*X[:, 0]*X[:, 1]
def gen_data(n, a, b, c, d, e, eps):
    if Xdist == 'U':
        rng = ss.uniform(loc=-4, scale=8)
    if Xdist == 'N':
        rng = ss.norm()
    X = rng.rvs(size=(n, 2))
    y = gen_model(X=X, a=a, b=b, c=c, d=d, e=e)
    if eps>0:
        y += ss.norm(loc=0, scale=eps).rvs(size=(n, ))
    y = y[:, np.newaxis]
    return X, y

# def get_mars_params(x, y):
#     param_grid = {"max_terms": [1, 2, 3, 5, 10], "max_degree": [1, 2, 3, 4, 5]}
#     mars = Earth()
#     sh = HalvingGridSearchCV(mars, param_grid, cv=3, factor=3, n_jobs=1).fit(x, y)
#     return sh.best_params_

def get_svr_params(x, y):
    param_grid = {"C": [0.1, 1, 10, 100, 300, 500, 750, 1000, 1500, 2000, 3000], "degree": [1, 2, 3, 4],
                  "epsilon": [0.01, 0.1, 1, 10]}
    svr = SVR(kernel="rbf")
    sh = HalvingGridSearchCV(svr, param_grid, cv=5, factor=3, n_jobs=1).fit(x, y)
    return sh.best_params_

def get_bmr_params(x, y, M, degree, substitution_policy="global"):
    n_trial = x.shape[0]
    param_grid = {
        "epsilon": [0.1, 0.2, 0.3, 0.5, 0.75, 1.0, 2.0, 5.0],
        "min_n_pts": [2, 5, 10, 25, 50, 100],
    }
    bmr = BMR(min_n_pts=10, M=M, substitution_policy=substitution_policy, degree=degree, epsilon=0.5)
    sh = HalvingGridSearchCV(bmr, param_grid, cv=3, factor=3, n_jobs=1).fit(x, y)
    params = sh.best_params_
    params['M'] = M
    params['substitution_policy'] = substitution_policy
    return params

In [4]:
# generate points in which prediction is made
grid_points = np.arange(-3, 3.1, 0.2)
mesh_X, mesh_Y = np.meshgrid(grid_points, grid_points)
mesh_pts = np.array([np.ravel(mesh_X), np.ravel(mesh_Y)]).transpose()

# Some diagnostics
1. In case of trully linear model with no noise expect coefficiants in all balls to be the same close to model values

In [ ]:
n=250
a=1
b=1
c=0
d=0
e=0
eps=0.00
Xdist = 'U'
X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
bmr = BMR(min_n_pts=10, M=2, epsilon=0.5)
bmr.fit(X, y)
# investigate the bmr structure
bmr.summary()

In [6]:
# check hyperparameters for BMR
# global substitution
n=250
a=1
b=1
c=0
d=0
e=0
eps=0.001
Xdist='U'
for loops in range(10):
    X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
    params = get_bmr_params(X, y, M=10, degree=1)
    print(params)

{'epsilon': 0.5, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.5, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.75, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.5, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.75, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.75, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.5, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.5, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.5, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}
{'epsilon': 0.5, 'min_n_pts': 2, 'M': 10, 'substitution_policy': 'global'}


In [ ]:
# nearest substitution
for loops in range(10):
    X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
    params = get_bmr_params(X, y, M=10, degree=1substitution_policy=substitution_policy="nearest")
    print(params)

In [25]:
def run_experiment(n, a, b, c, d, e, eps, mcloops=100):
    alpha = 0.05
    filename = f'CI_n={n}_a={a}_b={b}_c={c}_d{d}_e{e}_eps={eps}_X{Xdist}.csv'
    filename_params = f'CI_n={n}_a={a}_b={b}_c={c}_d{d}_e{e}_eps={eps}_X{Xdist}_hyperparams.'
    
    X_pred = mesh_pts
    y_true = gen_model(X_pred, a, b, c, d, e)

    #generate one sample to set method parameters
    X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
    #bmr_params = get_bmr_params(X, y, M=20, degree=1, substitution_policy='nearest')
    bmr_params = {'epsilon': 0.1, 'min_n_pts': 25, 'M': 10, 'substitution_policy': 'nearest'}
    #bmr_params = {'epsilon': 1, 'min_n_pts': 5, 'M': 10}
    #mars_params = get_mars_params(X, y[:, 0])
    svr_params = get_svr_params(X, y[:, 0])
    
    # save params to file
    #pickle.dump([bmr_params, mars_params, svr_params], open(filename_params, 'wb'))
    pickle.dump([bmr_params, svr_params], open(filename_params, 'wb'))
    
    # init methods
    #methods_labels = ['LR', 'BMR', 'MARS', 'SVR']
    methods_labels = ['LR', 'BMR', 'SVR']
    
    results = {}
    betas = {}
    intercepts = {}
    for method_label in methods_labels:
        results[method_label] = []
        betas[method_label] = []
        intercepts[method_label] = []

    for loop in range(mcloops):
        #if loop % 10 == 0:
        print(f'Running loop {loop}/{mcloops} for {filename}')
        
        # run all methods on new data set
        X, y = gen_data(n=n, a=a, b=b, c=c, d=d, e=e, eps=eps)
        #methods = [LinearRegression(), BMR(**bmr_params), Earth(**mars_params), SVR(**svr_params)]
        methods = [LinearRegression(), BMR(**bmr_params), SVR(**svr_params)]
        for method_label, method in zip(methods_labels, methods):
            if method_label == 'SVR':
                method.fit(X, y[:, 0])
            else:
                method.fit(X, y)
            pred = method.predict(X_pred)
            if len(pred.shape) > 1:
                pred = pred[:, 0]
            results[method_label].append(pred)
            # save coefficients
            if method_label == 'BMR':
                beta, intercept = method.coefficients(X_pred)
                betas['BMR'].append(beta)
                intercepts['BMR'].append(intercept)
            if method_label == 'LR':
                betas['LR'].append(method.coef_)
                intercepts['LR'].append(method.intercept_)
    coeff = {}
    coeff['BMR_beta1_low'] = np.quantile(np.array(betas['BMR']), q=alpha/2, axis=0)[:, 0] 
    coeff['BMR_beta1_up'] = np.quantile(np.array(betas['BMR']), q=1-alpha/2, axis=0)[:, 0]
    coeff['BMR_beta1_len'] = np.array(coeff['BMR_beta1_up']) - np.array(coeff['BMR_beta1_low'])
    coeff['BMR_beta2_low'] = np.quantile(np.array(betas['BMR']), q=alpha/2, axis=0)[:, 1] 
    coeff['BMR_beta2_up'] = np.quantile(np.array(betas['BMR']), q=1-alpha/2, axis=0)[:, 1]
    coeff['BMR_beta2_len'] = np.array(coeff['BMR_beta2_up']) - np.array(coeff['BMR_beta2_low'])
    coeff['BMR_I_low'] = np.quantile(np.array(intercepts['BMR']), q=alpha/2, axis=0) 
    coeff['BMR_I_up'] = np.quantile(np.array(intercepts['BMR']), 1-alpha/2, axis=0) 
    coeff['BMR_I_len'] = np.array(coeff['BMR_I_up']) - np.array(coeff['BMR_I_low'])
    coeff['LR_beta1_low'] = np.quantile(np.array(betas['LR']), q=alpha/2, axis=0)[:, 0].tolist()*X_pred.shape[0]
    coeff['LR_beta1_up'] = np.quantile(np.array(betas['LR']), q=1-alpha/2, axis=0)[:, 0].tolist()*X_pred.shape[0]
    coeff['LR_beta1_len'] = np.array(coeff['LR_beta1_up']) - np.array(coeff['LR_beta1_low'])
    coeff['LR_beta2_low'] = np.quantile(np.array(betas['LR']), q=alpha/2, axis=0)[:, 1].tolist()*X_pred.shape[0]
    coeff['LR_beta2_up'] = np.quantile(np.array(betas['LR']), q=1-alpha/2, axis=0)[:, 1].tolist()*X_pred.shape[0]
    coeff['LR_beta2_len'] = np.array(coeff['LR_beta2_up']) - np.array(coeff['LR_beta2_low'])
    coeff['LR_I_low'] = np.quantile(np.array(intercepts['LR']), q=alpha/2, axis=0).tolist()*X_pred.shape[0]
    coeff['LR_I_up'] = np.quantile(np.array(intercepts['LR']), q=1-alpha/2, axis=0).tolist()*X_pred.shape[0]
    coeff['LR_I_len'] = np.array(coeff['LR_I_up']) - np.array(coeff['LR_I_low'])
       
    # collect the results and prepare the csv
    df0 = pd.DataFrame([mesh_pts[:, 0], mesh_pts[:, 1]]).transpose()
    df0.columns = ['x', 'y']
    dfs = [df0]
    for method_label in methods_labels:
        dat = np.array(results[method_label]).transpose()
        ci_low = np.quantile(dat, q=alpha/2, axis=1)
        ci_up = np.quantile(dat, q=1-alpha/2, axis=1)
        mse = np.mean((dat - y_true.reshape(-1,1))**2, axis=1)
        df = pd.DataFrame([ci_low, ci_up, ci_up-ci_low, mse]).transpose()
        df.columns = [f'{method_label}_CI_low', f'{method_label}_CI_up', f'{method_label}_CI_len', f'{method_label}_MSE']
        dfs.append(df)
    # add DataFrame containing coefficients
    dfs.append(pd.DataFrame(coeff))
    df = pd.concat(dfs, axis=1)
    df.to_csv(filename, index=False)
    return df, betas, coeff

In [26]:
df, betas, coeff = run_experiment(n=250, a=1, b=2, c=0, d=0, e=0, eps=0.01, mcloops=2)

Running loop 0/5 for CI_n=250_a=1_b=2_c=0_d0_e0_eps=0.01_XU.csv


KeyboardInterrupt: 

In [17]:
np.mean(coeff['BMR_beta1_low'])

0.2396934065492736

In [18]:
np.mean(coeff['BMR_beta1_up'])

1.2512525165961206

In [19]:
np.mean(coeff['BMR_beta2_low'])

0.9810481193635655

In [20]:
np.mean(coeff['BMR_beta2_up'])

2.0722289163653485

In [59]:
# # df, betas, coeff = run_experiment(n=500, a=0, b=0, c=1, d=1, e=0, eps=0.0001, mcloops=100)
# # {'epsilon': 5.0, 'min_n_pts': 2, 'M': 20}
# # NORMAL
# df[['x', 'y', 'BMR_low_beta1', 'BMR_up_beta1', 'BMR_low_beta2', 'BMR_up_beta2']]

In [60]:
# # df, betas, coeff = run_experiment(n=500, a=0, b=0, c=1, d=1, e=0, eps=0.0001, mcloops=100)
# # {'epsilon': 5.0, 'min_n_pts': 100, 'M': 20}
# # UNIFORM
# df[['x', 'y', 'BMR_low_beta1', 'BMR_up_beta1', 'BMR_low_beta2', 'BMR_up_beta2']]

In [61]:
# # df, betas, coeff = run_experiment(n=500, a=0, b=0, c=1, d=1, e=0, eps=0.0001, mcloops=100)
# # {'epsilon': 0.24970063583258964, 'min_n_pts': 150, 'M': 20}
# # UNIFORM
# df[['x', 'y', 'BMR_low_beta1', 'BMR_up_beta1', 'BMR_low_beta2', 'BMR_up_beta2']]

In [62]:
# # bmr_params = {'epsilon': 1, 'min_n_pts': 5, 'M': 10}
# # UNIFORM
# df[['x', 'y', 'BMR_low_beta1', 'BMR_up_beta1', 'BMR_low_beta2', 'BMR_up_beta2']]

In [65]:
# pd.DataFrame.from_dict(coeff)

In [106]:
parser = argparse.ArgumentParser()
parser.add_argument("--n", type=int, required=True, help="sample size")
parser.add_argument("--a", type=float, required=True, help="param a")
parser.add_argument("--b", type=float, required=True, help="param b")
parser.add_argument("--c", type=float, required=True, help="param c")
parser.add_argument("--d", type=float, required=True, help="param d")
parser.add_argument("--e", type=float, required=True, help="param e")
parser.add_argument("--eps", type=float, required=True, help="noise")
parser.add_argument("--M", type=int, required=True, help="number of MC loops")
parser.add_argument("--X", type=str, required=True, help="X distribution")
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] --f F
ipykernel_launcher.py: error: the following arguments are required: --f


SystemExit: 2

/home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
Xdist = args.X
if Xdist not in ['U', 'N']:
    raise ValueError(f'--X must be U or N. Found {Xdist}')

run_experiment(n=args.n, a=args.a, b=args.b, c=args.c, d=args.d, e=args.e, eps=args.eps, mcloops=args.M)